In [1]:
from copy import deepcopy
import numpy as np
import pandas as pd
import calendar
from pybliometrics.scopus import ScopusSearch
from pybliometrics.scopus import AbstractRetrieval

from tqdm import tqdm

pd.options.display.max_columns = 30

filename = "./sample.txt"

In [2]:
### language
dic_language = {'eng': "English",
                'kor': "Korean"
               }


In [4]:
df_sample = pd.read_pickle("./df_sample.pkl")
print(df_sample.shape)
df_sample.head(3)

(1028, 34)


,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,affiliation_city,affiliation_country,author_count,author_names,author_ids,...,issn,source_id,eIssn,aggregationType,volume,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,fund_acr,fund_no,fund_sponsor
0,2-s2.0-85125145796,10.1016/j.egyr.2022.02.088,S2352484722003353,None,Data driven approach to forecast the next day ...,ar,Article,Fentis A.,60017798;127756529;116607338,Faculté des Sciences et Techniques de Mohammed...,Mohammedia;Mohammedia;Casablanca,Morocco;Morocco;Morocco,5,"Fentis, Ayoub;Rafik, Mohamed;Bahatti, Lhoussai...",57195404384;57208745688;55837734100;6506839198...,...,None,21100389511,23524847,Journal,8,None,None,3221-3233,Photovoltaic (PV) power has became an attracti...,Machine learning | PV power forecasting | Rene...,0,1,None,undefined,None
1,2-s2.0-85123676147,10.1016/j.egyr.2022.01.105,S2352484722001056,None,Energy modeling and photovoltaics integration ...,ar,Article,Al Huneidi D.I.,60113885,"Hamad Bin Khalifa University, College of Scien...",Doha,Qatar,3,"Al Huneidi, Dana I.;Tahir, Furqan;Al-Ghamdi, S...",57389610600;57201605786;56439247500,...,None,21100389511,23524847,Journal,8,None,None,166-171,Due to Qatar's increasing population and elect...,Climate change | Demand side management | Desi...,1,1,QNRF,NPRP12S-0212-190073,Qatar National Research Fund
2,2-s2.0-85124409874,10.1016/j.engappai.2022.104707,S0952197622000288,None,Multi-quantile recurrent neural network for fe...,ar,Article,Zhang X.Y.,60020595;60001455,"Royal Holloway, University of London;Anhui Uni...",Egham;Hefei,United Kingdom;China,3,"Zhang, Xiao Yu;Watkins, Chris;Kuenzel, Stefanie",57239241400;57447667900;55875409300,...,09521976,24182,None,Journal,110,None,104707,None,The purpose of feeder-level energy disaggregat...,Behind-the-meter PV generation | Deep neural n...,0,0,None,undefined,None


In [29]:
columns=["eid", "PT", "AU", "AF", "TI", 
         "SO", "SO_abb", "LA", "DT", "DE", 
         "ID", "AB", "C1", "RP", "EM", 
         "CR", "NR", "TC", "Z9", "SN", 
         "J9", "JI", "PD", "PY", "VL", 
         "AR", "DI", "SC"]

df_ab = pd.DataFrame(columns=columns)

In [32]:
starting_index = 0
data_index = list(range(starting_index, df_sample.shape[0]))

for art in tqdm(data_index, desc="looping over df_sample"):

    eid = df_sample.loc[art, "eid"]
    ab = AbstractRetrieval(eid, view="FULL")

    # 1. PT: publication type
    docu_type_ = ab.srctype

    # 2. AU: index name
    index_name_ = [a.indexed_name for a in ab.authors] if ab.authors else [None]

    # 3. AF: author name
    author_name_ = [f"{a.surname}, {a.given_name}" for a in ab.authors] if ab.authors else [None]

    # 4. TI: document title
    docu_title_ = df_sample.loc[art, "title"]

    # 5. SO: publication name
    src_title_ = ab.publicationName

    # 5-1. publication abbr.
    src_abb_ = ab.sourcetitle_abbreviation

    # 6. LA : Language
    try:
        language_ = dic_language[ab.language]
    except:
        language_ = "unknown"

    # 7. DT : Document Type
#     docu_type_ = ab.subtype
    docu_type_ = df_sample.loc[art, "subtype"]

    # 8. DE : Author Keywords
    auth_kw_ = ab.authkeywords

    if not auth_kw_:
        auth_kw_ = 'None'
    else:
        auth_kw_ = '; '.join(auth_kw_)

    # 9. ID : Keyword Plus
    kw_plus_ = 'None'

    # 10. AB : Abstract
    abstract_ = ab.abstract

    # 11. C1 : Author Address
#     if ab.authorgroup:
#         tmp = pd.DataFrame(ab.authorgroup)
#         grouped = tmp.groupby('organization').agg(lambda x: list(x))

#         aff_names = [str(aff) for aff in grouped["affiliation_id"].index.tolist()]
#         aff_ids = grouped["affiliation_id"].tolist()
#         citys = grouped["city"].tolist()
#         countrys = grouped["country"].tolist()
#         auids = grouped["auid"].tolist()
#         indexed_names = grouped["indexed_name"].tolist()

#         address = []
#         for aff_name, aff_id, city, country in zip(aff_names, aff_ids, citys, countrys):
#             if isinstance(aff_id, list):
#                 aff_id, city, country = aff_id[0], city[0], country[0]
#             address.append(f"{aff_id}, {aff_name}, {city}, {country}")

#         addresss_ = list(zip(auids, indexed_names, address))
    if ab.affiliation and ab.authors:
        df_aff = pd.DataFrame(ab.affiliation)
        df_aff["id"] = df_aff["id"].astype(str)
        df_auth = pd.DataFrame(ab.authors)
        df_authgroup = df_auth.explode("affiliation").groupby("affiliation").agg(list).reset_index()

        df_aff = pd.merge(df_aff, df_authgroup, left_on="id", right_on="affiliation").drop(["affiliation"], axis=1)
        df_aff["address"] = df_aff[['id', 'name', 'city', 'country']].apply(lambda x: ', '.join(x.astype(str)), axis=1)
        addresss_ = df_aff[["auid", "indexed_name", "address"]].values.tolist()
    else:
        addresss_ = []

    # df_aff = pd.DataFrame(ab.authorgroup)
    # grouped = df_aff.groupby("affiliation_id")
    # grouped[["organization", "affiliation_id", "city", "country"]]

    # 12. RP : Reprint Address
    rep_addr_ = "None"

    # 13. EM : E-mail Address
    em_addr_ = "None"

    # 14. CR : Cited References
    refs_ = []
    if ab.references != None:
        tmp = pd.DataFrame(ab.references)
        refcount = int(ab.refcount)

        for i in range(refcount):
            tmp_ = tmp.iloc[i]
            tmp_authors = tmp_['authors']
            if tmp_authors == None:
                tmp_authors = "[Anonymous]"
            tmp_year = tmp_['publicationyear']
            tmp_src = tmp_['sourcetitle']
            tmp_vol = tmp_['volume']
            tmp_page = tmp_['first']
            tmp_doi = tmp_['doi']

            ref = tmp_authors
            for item in [tmp_year, tmp_src, tmp_vol, tmp_page]:
                if item != None:
                    ref = ', '.join([ref, item])
            if tmp_doi != None:
                ref = ref + f", DOI {tmp_doi}"

            if i == 0:
                refs_.append(f"CR {ref}")
            else:
                refs_.append(f"   {ref}")

    # 15. NR : Cited Reference Count
    nr_ = ab.refcount

    # 16. TC : Web of Science Core Collection Times Cited Count
    tc_ = ab.citedby_count

    # 17. Z9 : Total Times Cited Count
    cc_ = tc_

    # 18. U1 : Usage Count (Last 180 Days)
    # 19. U2 : Usage Count (Since 2013)
    # 20. PU : Publisher = ELSEVIER SCI LTD
    # 21. PI : Publisher City = OXFORD
    # 22. PA : Publisher Address = THE BOULEVARD, LANGFORD LANE, KIDLINGTON, OXFORD OX5 1GB, OXON, ENGLAND
    # 23. SN : International Standard Serial Number (ISSN) = 0959-6526
    sn_ = "None" if ab.issn == None else ab.issn

    # 24. EI : Electronic International Standard Serial Number (eISSN) = 1879-1786
    # 25. J9 : 29-Character Source Abbreviation = J CLEAN PROD
    j9_ = ab.sourcetitle_abbreviation if ab.sourcetitle_abbreviation != None else "None"
    j9_ = j9_.upper()

    # 26. JI : ISO Source Abbreviation = J. Clean Prod.
    ji_ = ab.sourcetitle_abbreviation if ab.sourcetitle_abbreviation != None else "None"

    # 27. PD : Publication Date = JUL 1
    month = ab.coverDate.split('-')[1]
    date = ab.coverDate.split('-')[2]
    pd_ = f"{calendar.month_name[int(month)][:3].upper()} {int(date)}"

    # 28. PY : Publication Year = 2020
    py_ = ab.coverDate.split('-')[0]

    # 29. VL : Volumn = 260
    vl_ = ab.volume

    # 30. AR : Article Number = 121059
    ar_ = df_sample.loc[art, "article_number"] if df_sample.loc[art, "article_number"] != None else "None"

    # 31. DI : Digital Object Identifier = 10.1016/j.jclepro.2020.121059
    doi_ = ab.doi

    # 32. PG : Page Count = 14
    # 33. WC : Web of Science Categories = Green & Sustainable Science & Technology; Engineering, Environmental; Environmental Sciences
    # 34. SC : Research Areas = Science & Technology - Other Topics; Engineering; Environmental Sciences & Ecology
    if ab.subject_areas:        
        tmp = pd.DataFrame(ab.subject_areas)
        tmp_ = tmp["area"].tolist()
        sc_ = "; ".join(tmp_)
    else: 
        sc_ = [None]

    # 35. GA : Document Delivery Number = LL4XH
    # 36. UT : Accession Number = WOS:000531559900003
    # 37. DA : Date this report was generated. = 2020-06-14
    
    # summation
    data=[eid, docu_type_, index_name_, author_name_, docu_title_, 
                                src_title_, src_abb_, language_, docu_type_, auth_kw_, 
                                kw_plus_, abstract_, addresss_, rep_addr_, em_addr_, 
                                refs_, nr_, tc_, cc_, sn_, 
                                j9_, ji_, pd_, py_, vl_, 
                                ar_, doi_, sc_]
    
    df_ab_tmp = pd.DataFrame(dict(zip(columns, [[d] for d in data])))
    df_ab = pd.concat([df_ab, df_ab_tmp], axis=0)

looping over df_sample: 100%|██████████| 1028/1028 [00:17<00:00, 58.24it/s]


In [37]:
df_ab.reset_index(drop=True).to_pickle("df_ab.pkl")
df_ab.shape

(1028, 28)